In [1]:
import audio_processing as ap
import audio_utils as au
import math
import numpy.fft as fft
import matplotlib.pyplot as plt
%matplotlib inline

# CS 328 Final Project
## Thomas Bertschinger and Sanders McMillan
### Lab Notebook

### Tuesday, April 26, 2016 **(joint entry)**

Our final project will (likely) be *model* focused rather than experiment focused. We will "implemenet computational model(s) and compare results to existing human data." In this case, the human data will be musical compositions created by humans (*e.g.* Bach fugues) and our model will be a system that "composes" music (perhaps learning from human compositions). 

We set up a bibliography using Latex and BibTex, and a git repository to keep track of our code and materials. 

https://github.com/bertschingert/cs328-final/tree/master/references

### Wednesday, April 27, 2016 (joint entry)

We are thinking about doing a signal-processing based model that creates a representation of a signal and can classify the signal into categories such as instrument family, voice, etc. We will need to create a representation of audio sound in a (hopefully) small number of dimensions that includes enough relevant information so that we can classify a sound into a category such as brass versus string instrument, for example. 

The representation will likely include attributes such as attack and decay time of the waveform; the spectrum at various points in time; how much the spectrum changes over time; irregularities in the spectrum. 

Our model will ideally be able to classify instruments correctly at different amplitudes and pitches. It would also be important to limit the model to audio information that humans can actually percieve. (For example, it wouldn't make sense for our model to take into account frequences substantially above 20,000Hz because humans cannot hear that high.) 

Being able to pull out the relevant information from an audio signal is important because it will help us understand how humans can do things such as distinguish different people's voices. We know probably hundreds of different voices that we can identify from only a few words of speech. This is also important for being able to recognize different instruments present in a single audio signal. 

We think it would be plausible to create a neural network to identify different audio signals. We will first take an audio signal and use tools such as the discrete Fourier transform to create a suitable representation of the signal that omits extraneous information or information that humans cannot perciever. Then, we train a neural network to be able to identify, from the features of the representation, what category the signal belongs to. 

Theoretically, our model will be grounded in the similarity models learned earlier in this course, in addition to Gibsonian and Gestalt principles of perceiving invariants in stimuli and grouping similar and proximal stimuli as belonging to the same perceptual unit (e.g. being able to classify a signal as being of a certain category regardless of it's amplitude and signal, and classifying/perceiving similar successive waveforms as being from the same instrument). It will also involve the place and time theories for how humans transform air pressure hitting the ear into an auditory representation (which is where our Fourier transform and representation stages come in). 

### Monday, May 16, 2016 (joint entry)

We have written some code to do basic audio processing (FFT). We are also starting to create the NN..

In order to keep the neural network consistent, we will have the option to save weights and biases to a text file so that they can be loaded. 

In [2]:
left, right = au.read_raw_stereo("audio_files/violin-a440.raw")
chunk = left[:44100]
freqs = fft.rfft(chunk)
au.graph_fft(freqs[:10000])

FileNotFoundError: [Errno 2] No such file or directory: 'audio_files/violin-a440.raw'

We also started compiling our audio sample library. The audio samples are 1.5 second long clips downloaded from http://www.philharmonia.co.uk/explore/make_music. So far we have 5 different instruments (guitar, saxophone, flute, violin, and trumpet), each at three different pitches (A, C, and E). The octaves of the pitches are different for each instrument as the library did not contain all octaves for each instrument, and each instrument has different pitch restraints.

### Tuesday, May 17, 2016 (joint entry)

We are starting to write code that uses the FFT to get some information on the signal, such as attack time. This will be helpful for our representation. We are also starting to implement the neural network. 

### Saturday, May 21, 2016

Created the Hann window function. 

In [ ]:
test_signal = []
for i in range(100):
    test_signal.append(1)
w = ap.hann_window(test_signal)
au.graph_signal(w)

Created code to compute the spectral centroid

In [ ]:
wave = au.create_sine_wave(440, 1, 1)
spectrum = fft.rfft(wave)
print("before window: spectral centroid is ", ap.get_spectral_centroid(spectrum, 44100))
au.graph_fft(spectrum[:500])

In [ ]:
w_signal = ap.hann_window(wave)
w_spectrum = fft.rfft(w_signal)
print("after window: spectral centroid is ", ap.get_spectral_centroid(w_spectrum, 44100))
au.graph_fft(w_spectrum[:500])

### Sunday, May 22, 2016

Now using the python wave library to read .wav files. Created the function read_wav_mono in audio_utils.py which returns a list of the samples. 

In [ ]:
f = au.read_wav_mono('audio_files/guitar_A4_very-long_forte_normal.wav')
au.graph_signal(f)

In [ ]:
step= int(44100 / 4)
start = 44100
chunk = f[start:start+step]
s = fft.rfft(chunk)
au.graph_fft(s, 1000)

In [ ]:
w = ap.hann_window(chunk)
s = fft.rfft(w)
au.graph_fft(s, 1000)

In [ ]:
s = ap.spectral_flux(f)
au.graph_signal(s)

In [ ]:
f2 = au.read_wav_mono('audio_files/saxophone_A4_15_forte_normal.wav')
au.graph_signal(f2)

In [ ]:
s2 = ap.spectral_flux(f2)
au.graph_signal(s2)
for i in s2:
    print(i)

### Thursday, May 26

We finished writing code for our neural network, and are now testing and de-bugging it. Our neural network has an initialize_network function that initializes the weights and nodes of the networks based on input length, the output length, the number of hidden units, and the number of layers of the network. It also has a set_hidden_units function that allows you to set the number of hidden units at any particular hidden layer, and changes the weights accordingly.

In [ ]:
# Testing the initialize network function
import neural_net as nn
import numpy as np
nn.initialize_network(4, 3, 2, 3)

In [ ]:
#Testing the set hidden units function
nn.set_hidden_units(1, 5)

In [ ]:
#Testing the update weights function
input = np.array([[1,2,3, 4], [5, 6, 7, 8], [9,10,11, 12]])
output = np.array([[1, 0, 0], [0,1,0], [0,0,1]])
np.array(output)
nn.update_weights(input.T, output.T, 0.01)
print(nn.get_weights())

In [ ]:
(weights, bias) = nn.train_network(input.T, output.T, nn.update_weights, 25, 0.1)
print(weights)

In [ ]:
#Testing predict network function
inp = np.array([[1,2,3, 4]])
print(nn.predict_network(inp.T, nn.logistic))

In [ ]:
zcr_data = [np.loadtxt("reps/clarinet_zcr.txt"), np.loadtxt("reps/flute_zcr.txt"), np.loadtxt("reps/guitar_zcr.txt"), np.loadtxt("reps/saxophone_zcr.txt"), np.loadtxt("reps/violin_zcr.txt")]
sc_data = [np.loadtxt("reps/clarinet_centroid.txt"), np.loadtxt("reps/flute_centroid.txt"), np.loadtxt("reps/guitar_centroid.txt"), np.loadtxt("reps/saxophone_centroid.txt"), np.loadtxt("reps/violin_centroid.txt")]
h_data = []

instrs = ["guitar", "clarinet", "flute", "saxophone", "violin"]
ind = 0
for instr in instrs:
    h_data.append([])
    infile = open("reps/" + instr + "_harmonics.txt", 'r')
    for line in infile:
        ex = []
        for i in line.split(','):
            ex.append( float(i.strip()) )
        h_data[ind].append(ex)
    ind += 1
        
#print(h_data)
inp = []
inputs = []
outputs = []
for i in range(50):
    for j in range(5):
        inputs = []
        output = [0]*5
        #inputs.append(zcr_data[j][i])
        inputs.append(h_data[j][i])
        output[j] = 1
        outputs.append(output)
        inp.append(np.array(h_data[j][i]))
        # for k in range(5):
        #    if k != j:
         #       outputs[k].append(0)
# print(inp)
inp = np.array(inp)
print(inp)
output = np.array(outputs)
print(output)
test_inp = np.array([inp[2]])
nn.initialize_network(inp.T.shape[0], output.T.shape[0], 2, 5)
nn.train_network(inp.T, output.T, nn.update_weights, 50, 0.01)
real = nn.predict_network(inp.T, nn.logistic)
cm = get_confusion_matrix(real.T, output, output.T.shape[0])
print(cm)

In [ ]:
#Testing the neural network on the data from ps6

#copied from multilayernetwork.py 
def load_data(filename):
    data = np.loadtxt(filename,delimiter=',')
    inputs = []
    outputs = []
    for line in data:
        inputs.append(line[:-1])
        output = [0] * 10
        output[int(line[-1])] = 1
        outputs.append(output)
    return (np.array(inputs), np.array(outputs))

def get_confusion_matrix (actual, desired, outp_length):
    """Gets confusion matrix for a networks output."""
    confusion = np.zeros((outp_length, outp_length))
    for indx in range(len(actual)):
        # print(desired[indx])
        i = desired[indx].tolist().index(1)
        # print(actual[indx])
        j = actual[indx].argmax()
        confusion[i][j] += 1
    return confusion

(inp, out) = load_data("data/optdigits.tra")

print(nn.initialize_network(inp.T.shape[0], out.T.shape[0], 2, 10))
nn.train_network(inp.T, out.T, nn.update_weights, 50, 0.01)
actual = nn.predict_network(inp.T, nn.logistic)
cc = get_confusion_matrix(actual.T, out, out.T.shape[0])
print(cc)


### Thursday, May 26

Our neural network code is not predicting correctly, and I am spending my time going back to try and de-bug it.